### This file computes the correlation metrics of the appropriateness corpus and the UKP corpus (table 3 in the paper)

In [ ]:
import glob
import pandas as pd
import numpy as np
import editdistance
from scipy import stats
import math

from os import listdir
from collections import Counter
from os.path import isfile, join
from sklearn.model_selection import StratifiedKFold
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.pipeline import Sentencizer

pd.set_option('display.max_colwidth', None)

#### Reproduce results from the Argument Theory vs. Practice paper

In [ ]:
data_dir = '../../data/'
ukp2_dir = '../../repo/emnlp2016-empirical-convincingness/data/CSV-format/'

In [ ]:
df_dagstuhl = pd.read_csv(data_dir+'dagstuhl-15512-argquality-corpus-v2/dagstuhl-15512-argquality-corpus-v2-mean.csv')
df_sources = pd.read_csv(data_dir+'appropriateness-corpus/annotation_dataset_types_sourceids.csv')
df_appropriateness = pd.read_csv(data_dir+'appropriateness-corpus/appropriateness_corpus_mean.csv')

In [ ]:
df_appropriateness['source_id'] = df_sources['id'].tolist()

In [ ]:
df_appropriateness = df_appropriateness[df_appropriateness['source_id'].isin(df_dagstuhl['#id'].tolist())]

In [ ]:
df_appropriateness.sort_values('source_id', inplace=True)

In [ ]:
df_appropriateness.reset_index(inplace=True)

In [ ]:
rel_files_ukp2 = glob.glob(ukp2_dir+'*')
dfs = []
for file in rel_files_ukp2:
    if 'LICENSE.txt' not in file:
        tmp_df = pd.read_csv(file, sep='\t', names=['pair_id','gold_label','more_conv_arg','less_conv_arg'])
        tmp_df['issue'] = file.split('/')[-1].split('.csv')[0].split('.xml')[0].split('_')[0]
        tmp_df['stance'] = file.split('/')[-1].split('.csv')[0].split('.xml')[0].split('_')[1]
        dfs.append(tmp_df)
df_ukp2 = pd.concat(dfs)

In [ ]:
df_ukp2['less_conv_arg_id'] = df_ukp2['pair_id'].apply(lambda x: x.split('_')[0])
df_ukp2['more_conv_arg_id'] = df_ukp2['pair_id'].apply(lambda x: x.split('_')[1])

In [ ]:
df_ukp2 = df_ukp2[(df_ukp2['less_conv_arg_id'].isin(df_dagstuhl['#id'].tolist())) &(df_ukp2['more_conv_arg_id'].isin(df_dagstuhl['#id'].tolist()))]
df_ukp2.reset_index(inplace=True)

In [ ]:
def closest_by_edit(x):
    pair_id = x[0]
    arg_id1 = pair_id.split('_')[0]
    arg_id2 = pair_id.split('_')[1]
    x_conv_arg = x[1]
    arg_text1 = df_dagstuhl[df_dagstuhl['#id']==arg_id1]['argument'].tolist()[0]
    arg_text2 = df_dagstuhl[df_dagstuhl['#id']==arg_id2]['argument'].tolist()[0]
    dist1 = editdistance.eval(arg_text1,x_conv_arg)
    dist2 = editdistance.eval(arg_text2,x_conv_arg)
    if dist1 < dist2:
        return arg_id1
    else:
        return arg_id2

In [ ]:
df_ukp2['less_conv_arg_id'] = df_ukp2[['pair_id','less_conv_arg']].apply(lambda x: closest_by_edit(x), axis=1)
df_ukp2['more_conv_arg_id'] = df_ukp2[['pair_id','more_conv_arg']].apply(lambda x: closest_by_edit(x), axis=1)

In [ ]:
unique_labels = []
for labels in df_ukp2.gold_label.tolist():
    labels_list = labels.split(',')
    for label in labels_list:
        unique_labels.append(label)

In [ ]:
unique_labels = sorted(list(set(unique_labels)))

In [ ]:
MORE_CONVINCING_LABELS = [
    "o8_1", "o8_4", "o8_5", 
    "o9_1", "o9_2", "o9_3", "o9_4"
]
LESS_CONVINCING_LABELS = list(set(unique_labels)-set(MORE_CONVINCING_LABELS))

In [ ]:
unique_labels_dict = {x: i for i, x in enumerate(unique_labels)}

In [ ]:
unique_args = list(set(df_ukp2.less_conv_arg_id.tolist()+df_ukp2.more_conv_arg_id.tolist()))

In [ ]:
label_counts = {x: np.zeros(len(df_ukp2)) for x in unique_labels}

In [ ]:
for i, row in df_ukp2.iterrows():
    for label in unique_labels:
        if label in row.gold_label.split(','):
            label_counts[label][i] = 1
        else:
            label_counts[label][i] = 0

In [ ]:
for key, value in label_counts.items():
    df_ukp2[key] = value

In [ ]:
rel_dims_dagstuhl = list(set(df_dagstuhl.columns)-set(['argument', '#id', 'issue']))
rel_dims_appropriateness = ['Inappropriateness', 'Toxic Emotions', 'Excessive Intensity', 'Emotional Deception', 'Missing Commitment', 'Missing Seriousness', 'Missing Openness', 'Missing Intelligibility', 'Unclear Meaning', 'Missing Relevance', 'Confusing Reasoning', 'Other Reasons', 'Detrimental Orthography', 'Reason Unclassified']

In [ ]:
label_counts_more = {x: np.zeros(len(df_ukp2)) for x in rel_dims_dagstuhl}
label_counts_less = {x: np.zeros(len(df_ukp2)) for x in rel_dims_dagstuhl}

In [ ]:
for i, row in df_ukp2.iterrows():
    more_conv_values = df_dagstuhl[df_dagstuhl['#id']==row['more_conv_arg_id']][rel_dims_dagstuhl].values[0]
    less_conv_values = df_dagstuhl[df_dagstuhl['#id']==row['less_conv_arg_id']][rel_dims_dagstuhl].values[0]
    for j, label in enumerate(rel_dims_dagstuhl):
        label_counts_more[label][i] = more_conv_values[j]
        label_counts_less[label][i] = less_conv_values[j]

In [ ]:
for key, value in label_counts_more.items():
    df_ukp2['more_'+key] = value
for key, value in label_counts_less.items():
    df_ukp2['less_'+key] = value

In [ ]:
label_counts_more = {x: np.zeros(len(df_ukp2)) for x in rel_dims_appropriateness}
label_counts_less = {x: np.zeros(len(df_ukp2)) for x in rel_dims_appropriateness}

In [ ]:
for i, row in df_ukp2.iterrows():
    more_conv_values = df_appropriateness[df_appropriateness['source_id']==row['more_conv_arg_id']][rel_dims_appropriateness].values[0]
    less_conv_values = df_appropriateness[df_appropriateness['source_id']==row['less_conv_arg_id']][rel_dims_appropriateness].values[0]
    for j, label in enumerate(rel_dims_appropriateness):
        label_counts_more[label][i] = more_conv_values[j]
        label_counts_less[label][i] = less_conv_values[j]

In [ ]:
for key, value in label_counts_more.items():
    df_ukp2['more_'+key+'_app'] = value
for key, value in label_counts_less.items():
    df_ukp2['less_'+key+'_app'] = value

In [ ]:
corr_dict = {
    'rel_dims': rel_dims_dagstuhl, 
    'o8_1': [],
    'o8_4': [],
    'o8_5': [],
    'o9_1': [],
    'o9_2': [],
    'o9_3': [],
    'o9_4': [],
    'o5_1': [],
    'o5_2': [],
    'o5_3': [],
    'o6_1': [],
    'o6_2': [],
    'o6_3': [],
    'o7_1': [],
    'o7_2': [],
    'o7_3': [],
    'o7_4': [],
    'conv': [],
}


for rel_dim_dagstuhl in rel_dims_dagstuhl:
    for rel_dim_ukp in MORE_CONVINCING_LABELS+LESS_CONVINCING_LABELS:
        x1 = df_ukp2[df_ukp2[rel_dim_ukp]==1]['more_'+rel_dim_dagstuhl].tolist()
        x2 = df_ukp2[df_ukp2[rel_dim_ukp]==1]['less_'+rel_dim_dagstuhl].tolist()
        tieA = 0
        tieB = 0
        concordants = 0
        discordants = 0
        for i, x in enumerate(df_ukp2[rel_dim_ukp].tolist()):
            if x != 1:
                tieA+=1
        for a,b in zip(x1,x2):
            if a>b:
                concordants+=1
            elif b>a:
                discordants+=1
            else:
                tieB+=1
                
        all_ = concordants + discordants + 0 + tieB
        
        tau = (concordants - discordants) / math.sqrt((all_-0)*(all_-tieB))
        corr_dict[rel_dim_ukp].append(np.round(tau,2))
        
    x1 = df_ukp2['more_'+rel_dim_dagstuhl].tolist()
    x2 = df_ukp2['less_'+rel_dim_dagstuhl].tolist()
    tieA = 0
    tieB = 0
    concordants = 0
    discordants = 0

    for a,b in zip(x1,x2):
        if a>b:
            concordants+=1
        elif b>a:
            discordants+=1
        else:
            tieB+=1

    all_ = concordants + discordants + 0 + tieB

    tau = (concordants - discordants) / math.sqrt((all_-0)*(all_-tieB))
    corr_dict['conv'].append(np.round(tau,2))


In [ ]:
df_corr = pd.DataFrame(corr_dict)

In [ ]:
df_corr[['rel_dims','o8_1', 'o8_4', 'o8_5', 'o9_1', 'o9_2', 'o9_3', 'o9_4','o5_1', 'o5_2', 'o5_3', 'o6_1', 'o6_2', 'o6_3', 'o7_1', 'o7_2', 'o7_3', 'o7_4', 'conv']]

---

#### Use the same calculations to compute the correlations metrics for our corpus

In [ ]:
df_dagstuhl = pd.read_csv(data_dir+'dagstuhl-15512-argquality-corpus-v2/dagstuhl-15512-argquality-corpus-v2-mean.csv')
df_sources = pd.read_csv(data_dir+'appropriateness-corpus/annotation_dataset_types_sourceids.csv')
df_appropriateness = pd.read_csv(data_dir+'appropriateness-corpus/appropriateness_corpus_mean.csv')

In [ ]:
df_appropriateness.columns

In [ ]:
rel_files_ukp2 = glob.glob(ukp2_dir+'*')
dfs = []
for file in rel_files_ukp2:
    if 'LICENSE.txt' not in file:
        tmp_df = pd.read_csv(file, sep='\t', names=['pair_id','gold_label','more_conv_arg','less_conv_arg'])
        tmp_df['issue'] = file.split('/')[-1].split('.csv')[0].split('.xml')[0].split('_')[0]
        tmp_df['stance'] = file.split('/')[-1].split('.csv')[0].split('.xml')[0].split('_')[1]
        dfs.append(tmp_df)
df_ukp2 = pd.concat(dfs)

In [ ]:
df_ukp2.sample()

In [ ]:
df_ukp2['less_conv_arg_id'] = df_ukp2['pair_id'].apply(lambda x: x.split('_')[0])
df_ukp2['more_conv_arg_id'] = df_ukp2['pair_id'].apply(lambda x: x.split('_')[1])

In [ ]:
unique_args = list(set(df_ukp2.less_conv_arg_id.tolist()+df_ukp2.more_conv_arg_id.tolist()))

In [ ]:
df_appropriateness['source_id'] = df_sources['id'].tolist()

In [ ]:
df_appropriateness = df_appropriateness[df_appropriateness['source_id'].isin(unique_args)]

In [ ]:
df_appropriateness.sort_values('source_id', inplace=True)

In [ ]:
df_appropriateness.reset_index(inplace=True)

In [ ]:
df_ukp2.reset_index(inplace=True)

In [ ]:
def closest_by_edit(x):
    pair_id = x[0]
    arg_id1 = pair_id.split('_')[0]
    arg_id2 = pair_id.split('_')[1]
    x_conv_arg = x[1]
    arg_text1 = df_appropriateness[df_appropriateness['source_id']==arg_id1]['post_text'].tolist()[0]
    arg_text2 = df_appropriateness[df_appropriateness['source_id']==arg_id2]['post_text'].tolist()[0]
    dist1 = editdistance.eval(arg_text1,x_conv_arg)
    dist2 = editdistance.eval(arg_text2,x_conv_arg)
    if dist1 < dist2:
        return arg_id1
    else:
        return arg_id2

In [ ]:
df_ukp2['less_conv_arg_id'] = df_ukp2[['pair_id','less_conv_arg']].apply(lambda x: closest_by_edit(x), axis=1)
df_ukp2['more_conv_arg_id'] = df_ukp2[['pair_id','more_conv_arg']].apply(lambda x: closest_by_edit(x), axis=1)

In [ ]:
unique_labels = []
for labels in df_ukp2.gold_label.tolist():
    labels_list = labels.split(',')
    for label in labels_list:
        unique_labels.append(label)

In [ ]:
unique_labels = sorted(list(set(unique_labels)))

In [ ]:
MORE_CONVINCING_LABELS = [
    "o8_1", "o8_4", "o8_5", 
    "o9_1", "o9_2", "o9_3", "o9_4"
]
LESS_CONVINCING_LABELS = list(set(unique_labels)-set(MORE_CONVINCING_LABELS))

In [ ]:
unique_labels_dict = {x: i for i, x in enumerate(unique_labels)}

In [ ]:
label_counts = {x: np.zeros(len(df_ukp2)) for x in unique_labels}

In [ ]:
for i, row in df_ukp2.iterrows():
    for label in unique_labels:
        if label in row.gold_label.split(','):
            label_counts[label][i] = 1
        else:
            label_counts[label][i] = 0

In [ ]:
for key, value in label_counts.items():
    df_ukp2[key] = value

In [ ]:
rel_dims_appropriateness = ['Inappropriateness', 'Toxic Emotions', 'Excessive Intensity', 'Emotional Deception', 'Missing Commitment', 'Missing Seriousness', 'Missing Openness', 'Missing Intelligibility', 'Unclear Meaning', 'Missing Relevance', 'Confusing Reasoning', 'Other Reasons', 'Detrimental Orthography', 'Reason Unclassified']

In [ ]:
label_counts_more = {x: np.zeros(len(df_ukp2)) for x in rel_dims_appropriateness}
label_counts_less = {x: np.zeros(len(df_ukp2)) for x in rel_dims_appropriateness}

In [ ]:
for i, row in df_ukp2.iterrows():
    more_conv_values = df_appropriateness[df_appropriateness['source_id']==row['more_conv_arg_id']][rel_dims_appropriateness].values[0]
    less_conv_values = df_appropriateness[df_appropriateness['source_id']==row['less_conv_arg_id']][rel_dims_appropriateness].values[0]
    for j, label in enumerate(rel_dims_appropriateness):
        label_counts_more[label][i] = more_conv_values[j]
        label_counts_less[label][i] = less_conv_values[j]

In [ ]:
for key, value in label_counts_more.items():
    df_ukp2['more_'+key+'_app'] = value
for key, value in label_counts_less.items():
    df_ukp2['less_'+key+'_app'] = value

In [ ]:
corr_dict = {
    'rel_dims': rel_dims_appropriateness, 
    'o8_1': [],
    'o8_4': [],
    'o8_5': [],
    'o9_1': [],
    'o9_2': [],
    'o9_3': [],
    'o9_4': [],
    'o5_1': [],
    'o5_2': [],
    'o5_3': [],
    'o6_1': [],
    'o6_2': [],
    'o6_3': [],
    'o7_1': [],
    'o7_2': [],
    'o7_3': [],
    'o7_4': [],
    'conv': [],
}

for rel_dim_appropriateness in rel_dims_appropriateness:
    for rel_dim_ukp in MORE_CONVINCING_LABELS+LESS_CONVINCING_LABELS:
        x1 = df_ukp2[df_ukp2[rel_dim_ukp]==1]['more_'+rel_dim_appropriateness+'_app'].tolist()
        x2 = df_ukp2[df_ukp2[rel_dim_ukp]==1]['less_'+rel_dim_appropriateness+'_app'].tolist()
        tieA = 0
        tieB = 0
        concordants = 0
        discordants = 0
        for i, x in enumerate(df_ukp2[rel_dim_ukp].tolist()):
            if x != 1:
                tieA+=1
        for a,b in zip(x1,x2):
            if a>b:
                concordants+=1
            elif b>a:
                discordants+=1
            else:
                tieB+=1
                
        all_ = concordants + discordants + 0 + tieB
        
        tau = (concordants - discordants) / math.sqrt((all_-0)*(all_-tieB))
        corr_dict[rel_dim_ukp].append(np.round(tau,2))
        
    x1 = df_ukp2['more_'+rel_dim_appropriateness+'_app'].tolist()
    x2 = df_ukp2['less_'+rel_dim_appropriateness+'_app'].tolist()
    tieA = 0
    tieB = 0
    concordants = 0
    discordants = 0

    for a,b in zip(x1,x2):
        if a>b:
            concordants+=1
        elif b>a:
            discordants+=1
        else:
            tieB+=1

    all_ = concordants + discordants + 0 + tieB

    tau = (concordants - discordants) / math.sqrt((all_-0)*(all_-tieB))
    corr_dict['conv'].append(np.round(tau,2))



In [ ]:
df_corr = pd.DataFrame(corr_dict)

In [ ]:
df_corr[['rel_dims','o8_1', 'o8_4', 'o8_5', 'o9_1', 'o9_2', 'o9_3', 'o9_4','o5_1', 'o5_2', 'o5_3', 'o6_1', 'o6_2', 'o6_3', 'o7_1', 'o7_2', 'o7_3', 'o7_4', 'conv']]

In [ ]:
df_corr.T